<a href="https://colab.research.google.com/github/MridulMalvi/PyTorch-lib/blob/main/rnn_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

df = pd.read_csv('/content/Updated_590_Unique_QA_Dataset.csv')

df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [2]:
# tokenize
def tokenize(text):
  text = text.lower()
  text = text.replace('?','')
  text = text.replace("'","")
  return text.split()

In [3]:
tokenize('What is the capital of France?')

['what', 'is', 'the', 'capital', 'of', 'france']

In [4]:
# vocab
vocab = {'<UNK>':0}

In [5]:
def build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])

  merged_tokens = tokenized_question + tokenized_answer

  for token in merged_tokens:

    if token not in vocab:
      vocab[token] = len(vocab)


In [26]:
df.apply(build_vocab, axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
585,None
586,None
587,None
588,None


In [7]:
len(vocab)

404

In [8]:
# convert words to numerical indices
def text_to_indices(text, vocab):

  indexed_text = []

  for token in tokenize(text):

    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [9]:
text_to_indices("What is campusx", vocab)

[1, 2, 0]

In [10]:
import torch
from torch.utils.data import Dataset, DataLoader

In [11]:
class QADataset(Dataset):

  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):

    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [12]:
dataset = QADataset(df, vocab)

In [13]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [14]:
for question, answer in dataloader:
  print(question, answer[0])

tensor([[ 42, 125,   2,  62,  63,   3, 338, 339,   5,  73]]) tensor([340])
tensor([[ 10, 379, 380]]) tensor([381, 382])
tensor([[  1,   2,   3, 394,   5,  14, 345, 395]]) tensor([396])
tensor([[ 42, 125,   2,  62,  63,   3, 126, 127]]) tensor([128])
tensor([[19, 42, 43, 44, 45, 46, 47, 48]]) tensor([49])
tensor([[  1,   2,   3, 359, 360,   5, 361]]) tensor([362, 363, 364])
tensor([[10, 11, 67]]) tensor([377, 397])
tensor([[ 10,  11, 383]]) tensor([384, 191])
tensor([[ 10,  11,   3, 370]]) tensor([371, 372])
tensor([[  1,   2,   3, 221,   5, 222, 223, 224]]) tensor([225])
tensor([[ 42, 344,  55,   3, 345, 107]]) tensor([345, 346, 347])
tensor([[ 1,  2,  3, 37, 38, 39, 40]]) tensor([41])
tensor([[ 10,  11,   3, 370]]) tensor([371, 372])
tensor([[ 42,   2,   3, 354, 123,  19,   3,  45]]) tensor([355, 124])
tensor([[  1,   2,   3, 359, 360,   5, 361]]) tensor([362, 363, 364])
tensor([[ 10, 118, 328,   3, 186, 329, 330, 331,  19, 332, 333]]) tensor([334, 335])
tensor([[ 42, 344,  55,   3, 3

In [15]:
import torch.nn as nn

In [16]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [17]:
x = nn.Embedding(404, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 404)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 404])


In [18]:
learning_rate = 0.001
epochs = 20

In [19]:
model = SimpleRNN(len(vocab))

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [22]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # Pad answer sequences to the maximum length in the batch
    padded_answer = torch.nn.utils.rnn.pad_sequence(answer, batch_first=True, padding_value=vocab['<UNK>'])

    # forward pass
    # Adjust model output shape to match padded_answer shape
    output = model(question).unsqueeze(1).repeat(1, padded_answer.size(1), 1)


    # loss -> output shape (batch_size, sequence_length, vocab_size), target shape (batch_size, sequence_length)
    loss = criterion(output.view(-1, output.size(-1)), padded_answer.view(-1))


    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 1880.244934
Epoch: 2, Loss: 827.441729
Epoch: 3, Loss: 604.777508
Epoch: 4, Loss: 477.799570
Epoch: 5, Loss: 385.098449
Epoch: 6, Loss: 317.945959
Epoch: 7, Loss: 282.546427
Epoch: 8, Loss: 263.521075
Epoch: 9, Loss: 252.668084
Epoch: 10, Loss: 248.446619
Epoch: 11, Loss: 246.153166
Epoch: 12, Loss: 242.175101
Epoch: 13, Loss: 242.010522
Epoch: 14, Loss: 240.912974
Epoch: 15, Loss: 239.392894
Epoch: 16, Loss: 240.960159
Epoch: 17, Loss: 237.249053
Epoch: 18, Loss: 238.665683
Epoch: 19, Loss: 237.025582
Epoch: 20, Loss: 238.330702


In [23]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [24]:
predict(model, "What is the largest planet in our solar system?")

jupiter


In [25]:
list(vocab.keys())[7]

'paris'